In [2]:
import math
import numpy as np
import pandas as pd
from pprint import pprint
import pymongo

In [22]:
# Mongo client
client = pymongo.MongoClient("mongodb://localhost:27017")

In [23]:
# creating a "terrorism" database 
db = client["terrorism"]

In [24]:
client.list_database_names()

['MovieLens', 'admin', 'config', 'local', 'practice', 'student', 'terrorism']

In [25]:
# collection: in terrorism database
collection = db["incidents"]

#### location object $\rightarrow$ GeoObject 
location: {

      type: "Point",
      
      coordinates: [longitude, latitude]
}

In [42]:
# collections in the database
db.list_collection_names()

['incidents']

In [43]:
# creating index: 2dsphere (for location data)
db.incidents.create_index([("location", pymongo.GEOSPHERE)])

'location_2dsphere'

In [44]:
pymongo.GEOSPHERE

'2dsphere'

In [56]:
cursor = db.incidents.aggregate([
           {
             '$geoNear': {
                           'near': {'type': "Point", 'coordinates': [ 73.99279 , 40.719296 ]},
                           'spherical': True,
                           'distanceField': "dist.calculated",
                           'num': 5
                         }
           }, {

             '$project': {'_id': 0, 'location': 1, 'country_txt': 1, 'gname': 1}
           }
        ]);

In [57]:
# printing the events
for event in cursor:
    print(event)

{'country_txt': 'Kyrgyzstan', 'gname': 'Unknown', 'location': {'type': 'Point', 'coordinates': [72.981488, 40.933154]}}
{'country_txt': 'China', 'gname': 'Turkestan Islamic Party', 'location': {'type': 'Point', 'coordinates': [75.94243, 39.51953]}}
{'country_txt': 'China', 'gname': 'Uighur Separatists', 'location': {'type': 'Point', 'coordinates': [75.989761, 39.470107]}}
{'country_txt': 'China', 'gname': 'Uighur Separatists', 'location': {'type': 'Point', 'coordinates': [75.990067, 39.469409]}}
{'country_txt': 'China', 'gname': 'Uighur Separatists', 'location': {'type': 'Point', 'coordinates': [75.989641, 39.46616]}}


In [62]:
project = {'_id': 0, 'location': 1, 'country_txt': 1, 'gname': 1, 'weaptype1_txt': 1}
cursor = db.incidents.find(
                        {'country_txt': 'Nepal'}, 
                          project
                      ).limit(5)

In [63]:
for k in cursor:
    print(k)

{'country_txt': 'Nepal', 'gname': 'Unknown', 'weaptype1_txt': 'Explosives', 'location': {'type': 'Point', 'coordinates': [86.267047, 26.767787]}}
{'country_txt': 'Nepal', 'gname': 'Unknown', 'weaptype1_txt': 'Incendiary', 'location': {'type': 'Point', 'coordinates': [85.317684, 26.95183]}}
{'country_txt': 'Nepal', 'gname': 'Unknown', 'weaptype1_txt': 'Explosives', 'location': {'type': 'Point', 'coordinates': [82.09, 28.2]}}
{'country_txt': 'Nepal', 'gname': 'Unified Communist Party of Nepal (Maoist)', 'weaptype1_txt': 'Unknown', 'location': {'type': 'Point', 'coordinates': [85.329498, 27.708754]}}
{'country_txt': 'Nepal', 'gname': 'Unknown', 'weaptype1_txt': 'Firearms', 'location': {'type': 'Point', 'coordinates': [83.003611, 27.655162]}}
